In [1]:
import os
import torch
import shutil
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from point_e.util.plotting import plot_point_cloud
from point_e.models.download import load_checkpoint
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config

/home/noamatia/miniconda3/envs/point-e/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])
base_model.load_state_dict(load_checkpoint(base_name, device))

<All keys matched successfully>

In [3]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model],
    diffusions=[base_diffusion],
    num_points=[1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0],
    model_kwargs_key_filter=['texts'], 
)

In [8]:
experimental_ts = [1, 10, 20, 30, 40, 45, 50, 55, 60, 63]
prompt_pairs = [('a chair', 'an armchair'),
                ('a chair', 'a chair with long legs'),
                ('a chair', 'a chair with square backrest'),
                ('a table', 'a table with cylindrical legs'),
                ('a table', 'a table with a glass top'),
                ('a car', 'a car with big wheels'),
                ('a car', 'a car with a sunroof'),
                ('an airplane', 'an airplane with long wings'),
                ('an airplane', 'an airplane with a propeller'),
                ('a motorcycle', 'a motorcycle with a windshield')]

In [9]:
def plot_pc(sampler, samples, j, output_dir, experimental_t, prompt):
    pc = sampler.output_to_point_clouds(samples)[j]
    fig = plot_point_cloud(pc, color=False)
    fig.savefig(os.path.join(output_dir, f'{experimental_t}_{prompt.replace(" ", "_")}.png'))
    plt.close()

In [10]:
for prompt1, prompt2 in prompt_pairs:
    for experimental_t in experimental_ts:
        sampler.experimental_t = experimental_t
        for i in range(1):
            output_dir = os.path.join('experiment1', f'{prompt1.replace(" ", "_")}_{prompt2.replace(" ", "_")}_{i}')
            os.makedirs(output_dir, exist_ok=True)
            samples = None
            for x in tqdm(sampler.sample_batch_progressive(batch_size=2, model_kwargs=dict(texts=[prompt1, prompt2]))):
                samples = x
            for j, prompt in zip(range(2), [prompt1, prompt2]):
                plot_pc(sampler, samples, j, output_dir, experimental_t, prompt)
            for file in os.listdir('experimental_sampler'):
                shutil.move(os.path.join('experimental_sampler', file), 
                            os.path.join(output_dir, file))

65it [00:06,  9.80it/s]
65it [00:06,  9.78it/s]
65it [00:06,  9.74it/s]
65it [00:06,  9.70it/s]
65it [00:06,  9.65it/s]
65it [00:06,  9.63it/s]
65it [00:06,  9.41it/s]
65it [00:06,  9.69it/s]
65it [00:06,  9.71it/s]
65it [00:06,  9.69it/s]
65it [00:06,  9.69it/s]
65it [00:06,  9.69it/s]
65it [00:06,  9.68it/s]
65it [00:06,  9.67it/s]
65it [00:06,  9.66it/s]
65it [00:06,  9.65it/s]
65it [00:06,  9.65it/s]
65it [00:06,  9.64it/s]
65it [00:06,  9.61it/s]
65it [00:06,  9.61it/s]
65it [00:06,  9.63it/s]
65it [00:06,  9.62it/s]
65it [00:06,  9.62it/s]
65it [00:06,  9.61it/s]
65it [00:06,  9.59it/s]
65it [00:07,  9.26it/s]
65it [00:06,  9.60it/s]
65it [00:06,  9.61it/s]
65it [00:06,  9.61it/s]
65it [00:06,  9.60it/s]
65it [00:06,  9.63it/s]
65it [00:06,  9.62it/s]
65it [00:06,  9.57it/s]
65it [00:06,  9.62it/s]
65it [00:06,  9.59it/s]
65it [00:06,  9.60it/s]
65it [00:06,  9.60it/s]
65it [00:06,  9.60it/s]
65it [00:07,  9.17it/s]
65it [00:06,  9.59it/s]
65it [00:06,  9.62it/s]
65it [00:06,  9.

In [11]:
html = "<table>\n"
for prompt1, prompt2 in prompt_pairs:
    for experimental_t in experimental_ts:
        for i in range(1):
            output_dir = f'experiment1/{prompt1.replace(" ", "_")}_{prompt2.replace(" ", "_")}_{i}'
            col1 = f'{prompt1}'
            col2 = f'{prompt2}'
            col4 = f'{prompt1} (t={experimental_t})'
            html += f'<tr><td><font size="5">{col1}</font></td><td><font size="5">{col2}</font></td><td><font size="5">{col4}</font></td></tr>\n'
            col1_src = f'{output_dir}/{experimental_t}_{prompt1.replace(" ", "_")}.png'
            col2_src = f'{output_dir}/{experimental_t}_{prompt2.replace(" ", "_")}.png'
            col4_src = f'{output_dir}/{experimental_t}.png'
            html += f"<tr><td><img src='{col1_src}'></td><td><img src='{col2_src}'></td><td><img src='{col4_src}'></td></tr>\n"
html += "</table>"
with open('output.html', 'w') as f:
    f.write(html)